In [1]:
import pandas as pd
import numpy as np
from os.path import join, isfile

import matplotlib
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_auc_score, brier_score_loss, accuracy_score
from sklearn.calibration import calibration_curve

In [2]:
def load_labels(start, end, label_path, run_freq, file_format):
    labels = []
    for run_date in pd.date_range(start, end, freq=run_freq[0], closed="left"):
        file_name = join(label_path, f'track_step_NCARSTORM_d01_{run_date.strftime("%Y%m%d-0000")}.{file_format}')
        
        if isfile(file_name):
            if file_format == 'parquet':
                labels.append(pd.read_parquet(file_name))
            elif file_format == 'csv':
                labels.append(pd.read_csv(file_name))
        else:
            continue

    return pd.concat(labels)

In [3]:
labels = load_labels("2013-01-01", "2013-12-31", "/glade/campaign/mmm/parc/sobash/mode_objects_hwt/track_data_ncarstorm_3km_REFL_COM_hyst_csv", "daily", "csv")

cnn_training_data = pd.read_csv('/glade/work/sobash/HWT_mode/model_cnn_test2_addstorms2_noaugval_newtrain/predictions_train.csv') # all storms in hand labeled dataset
cnn_test_data = pd.read_csv('/glade/work/sobash/HWT_mode/model_cnn_test2_addstorms2_noaugval_newtrain/predictions_test.csv')

cnn_training_data = cnn_training_data.iloc[:1781] #only get first set of storms (rest are rotated)
hand_labeled = pd.concat([cnn_training_data, cnn_test_data])
#hand_labeled = hand_labeled.loc[hand_labeled['run_date'] >= "2013-01-01"]

# create columns with int and names
categories_int = {'Q1':0, 'Q2':0, 'S1':1, 'S2':0, 'S3':1, 'D1':2, 'D2':2}
categories = {'Q1':'QLCS', 'Q2': 'QLCS', 'S1':'Supercell', 'S2':'QLCS', 'S3':'Supercell', 'D1':'Disorganized', 'D2':'Disorganized'}

hand_labeled['new_label_int'] = hand_labeled['label'].apply(lambda x: categories_int[x])
hand_labeled['new_label'] = hand_labeled['label'].apply(lambda x: categories[x])

# create unique id for both datasets
hand_labeled['unique_id'] = hand_labeled['run_date'].astype(str) + '.' + hand_labeled['track_id'].astype(str) + '.' + hand_labeled['track_step'].astype(str)
labels['unique_id'] = labels['Run_Date'].astype(str).apply(lambda x: x[:10]) + '.' + labels['Track_ID'].apply(lambda x: str(x[-3:]).lstrip("0")) + '.' + labels['Duration'].astype(str)

# create matched storms dataframe
matched_labels = labels[labels['unique_id'].isin(hand_labeled.loc[:, 'unique_id'])] #extract only the storms with hand labels

# create five label columns in matched_labels dataframe
matched_labels.loc[:, 'true_label'] = hand_labeled.loc[:, 'new_label'].values.copy()
matched_labels.loc[:, 'true_label_int'] = hand_labeled.loc[:, 'new_label_int'].values.copy()
modes = ['QLCS', 'Supercell', 'Disorganized']
for mode in modes: matched_labels.loc[:, f'true_label_{mode}_int'] = np.where(matched_labels[f'true_label'] == mode, 1.0, 0.0)

/glade/work/sobash/conda-envs/tf2py37/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [10]:
# define the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', C=1, max_iter=5000)

# fit the model on the whole dataset
train_mask = matched_labels['Run_Date'] < '2013-06-25 00:00:00'
test_mask = ~train_mask

features = ['UP_HELI_MAX_max','major_axis_length','area']
X_train = matched_labels[features][train_mask].values
y_train = matched_labels['true_label_int'][train_mask].values
X_test  = matched_labels[features][test_mask].values
y_test = matched_labels['true_label_int'][test_mask].values

# train model with 2013 training dataset
model.fit(X_train, y_train)

# predict the class label for 2013 test dataset
yhat = model.predict_proba(X_test)

# output predictions if desired
#lr_predictions_df = pd.DataFrame(data=yhat, index=range(yhat.shape[0]), columns=['LR_QLCS_prob','LR_Supercell_prob','LR_Disorganized_prob'])
#lr_predictions_df['unique_id'] = matched_labels['unique_id'][1781:].values
#lr_predictions_df.to_pickle('lr_predictions_test_storms.pk')

In [12]:
# make LR Predictions for 2017-2019 storms
objects = load_labels("2016-01-01", "2019-12-31", "/glade/work/cbecker/WRF_all/track_data_hrrr_3km_csv_refl", "daily", "csv")
objects['unique_id'] = objects['Run_Date'].astype(str).apply(lambda x: x[:10]) + '.' + objects['Track_ID'].apply(lambda x: str(x[-3:]).lstrip("0")) + '.' + objects['Duration'].astype(str)

# predict the class label
yhat = model.predict_proba(objects[features].values)

# output predictions
lr_predictions_df = pd.DataFrame(data=yhat, index=range(yhat.shape[0]), columns=['LR_QLCS_prob','LR_Supercell_prob','LR_Disorganized_prob'])
lr_predictions_df['Run_Date'] = objects['Run_Date'].values
lr_predictions_df['unique_id'] = objects['unique_id'].values
lr_predictions_df.to_pickle('lr_predictions_2016-2019_storms.pk')